In [27]:
import pandas as pd
import numpy as np
from functools import reduce
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.feature_extraction import stop_words
 


data = pd.read_csv('complete_allbrands_users_captions_list.csv', encoding="latin-1")

In [28]:
data


,Unnamed: 0,username,followers_count,caption
0,0,50shadesofbondi,12569,['BRB! ? || @50shadesofbondi is taking 2 month...
1,1,88thatelier,15007,"['Our regular customer, Bella brings our bambo..."
2,2,_____bettina_____,21367,['Sunday is for snuggling and today we are clo...
3,3,_karmashek_,11328,"['? ? ??? ??????, ????????? ?????, ????? ?????..."
4,4,_laurennelson_,24401,['Saturdays in Central Park are the best. ???...
5,5,_neongypsy,17542,['Classic boho tunic dress in white ? dropping...
6,6,_swop,10890,"['Tag your crew ?????????', ""It's Saturday! Wh..."
7,7,a.a.k.s,15335,['All work and no play makes Akosua a dull gir...
8,8,abimfox,25502,['Congratulations @sydhayeshair on the launch ...
9,9,abstractioninaction,65027,"['Mariela Scafati, (Argentina, 1973), ""En busc..."


In [15]:


M_list = pd.read_csv('700 profiles for claissifier - second round.tsv', sep='\t')
M_list = M_list[M_list.consumer==1 ]
M_list['joined'] = 1

result = pd.merge(M_list, data, on='username', how='outer')



data = result[result.joined==1]
data = data.dropna(axis=0, how='any')
data = (data.drop_duplicates(subset=['username'], keep='first')).reset_index()


In [16]:
data

,index,profile_url,username,consumer,retailer,others,joined,Unnamed: 0,followers_count,caption
0,56,https://www.instagram.com/meireehrensperger/,meireehrensperger,1.0,0.0,0.0,1.0,241.0,10747.0,['Os encantos do mar para trazer boas energias...
1,204,https://www.instagram.com/my_bikini_musthaves/,my_bikini_musthaves,1.0,0.0,0.0,1.0,937.0,10595.0,"[""It's time to update your LBB again. I've sel..."
2,417,https://www.instagram.com/catwalkschoolgates/,catwalkschoolgates,1.0,0.0,0.0,1.0,10.0,12468.0,['Is Halloween ? after all saw this jacket @ch...
3,426,https://www.instagram.com/bikinisandbibs/,bikinisandbibs,1.0,0.0,0.0,1.0,64.0,10832.0,"[""Purchases?? Both been worn already! Pics com..."
4,436,https://www.instagram.com/thewearnecessities/,thewearnecessities,1.0,0.0,0.0,1.0,60.0,27161.0,"['Amanda Seyfried In Valentino', 'Amanda Seyfr..."
5,450,www.instagram.com/highheelprncess/,highheelprncess,1.0,0.0,0.0,1.0,716.0,24820.0,"['At @breederscup w @jewel #horse', '@thankyo..."
6,451,www.instagram.com/janekasemsarn/,janekasemsarn,1.0,0.0,0.0,1.0,817.0,19314.0,"['A little party never killed nobody ?', 'Need..."
7,455,www.instagram.com/offwithyourclothes/,offwithyourclothes,1.0,0.0,0.0,1.0,1377.0,26550.0,"['y a s @josefinehj', 'I c o n @christinecente..."
8,461,www.instagram.com/tutibarrera/,tutibarrera,1.0,0.0,0.0,1.0,1341.0,59165.0,['Happy Halloween ? #halloween #myhusbandtheph...
9,462,www.instagram.com/alicemapy/,alicemapy,1.0,0.0,0.0,1.0,66.0,18546.0,"['Schoolgirl, mermaid, chinese princess and ga..."


In [18]:
data = pd.read_csv('complete_allbrands_users_captions_list.csv', encoding="latin-1")
LeSir = pd.read_csv('emporiosirenuse_followers_accounts.csv', sep=',')


LeSir['joined'] = 1

result = pd.merge(LeSir, data, on='username', how='outer')



data = result[result.joined==1]
data = data.dropna(axis=0, how='any')
data = (data.drop_duplicates(subset=['username'], keep='first')).reset_index()


In [20]:
dftest = data

In [21]:
### Clean Text


# Milano

userlist=[]
emojis = []
usernameforeachpost  = []
f2post = []
f3post = []

captionsfromsample = dftest
#captionsfromsample.drop(['level_0', 'index', 'Unnamed: 0'], axis =1, inplace = True)

listof = []
for num, singlepost in enumerate(captionsfromsample.caption): 
    ## f1 separates by user. This is each user's total supply of posts
    f1 = singlepost.split("\', \'")

    # take only the first quarter of their text
    total= (len(f1)/4)
    f1 = f1[0:int(total)]

    f3 = []

    #for i in (range(len(f1))):
    #    userlist.append(dftest.username)"""
    
    for j,k in enumerate(f1):

        userlist.append(dftest.username[num])
        
        f2 = (f1[j].replace("\n", " ")).replace("\\", "")
        
        #get emojis
        emoji = re.findall(r'[^\w\s,]', f2)
        emojis.append(emoji)
        
        f2post.append(f2)
        f3post.append(f3)
        f3.append(f2)
        #usernameforeachpost.append(dftest.username[num])

        
print(len(f2), len(f2post), len(f3post), len(emojis), len(userlist))


2575 7432 7432 7432 7432


In [22]:
df1991 = pd.DataFrame({'caption': f2post, 'username': userlist, 'emojis': emojis}) 
df1991.to_csv('Sirenuse_follower_posts_pure.csv', sep=',')

In [26]:
len(df1991.username.unique())

104